In [2]:
import numpy as np

def normalize(v):
    ss = np.sum(v**2)
    return v / np.sqrt(ss)

def gaussian_kernel_to_dist(k, sigma):
    return np.sqrt(-2 * sigma**2 * np.log(k))


In [3]:
np.random.seed(123)
n, r = 5, 2

In [4]:
while True:
    Lambda = np.abs(np.random.normal(0, 1, (n, r)))
    Lambda = np.apply_along_axis(normalize, 1, Lambda)

    K = Lambda @ Lambda.T
    np.fill_diagonal(K, 1)

    D = np.vectorize(lambda k: gaussian_kernel_to_dist(k, 1))(K)

    MTM = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            MTM[i, j] = (D[0, j]**2 + D[i, 0]**2 - D[i, j]**2) / 2

    eigenvalues1 = np.linalg.eigvals(K)
    eigvals, eigvecs = np.linalg.eigh(MTM)
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    M = np.sqrt(np.diag(eigvals[:r+1])) @ eigvecs[:, :r+1].T
    thres = np.max(np.abs(M * (M < 0)))

    v = np.full((r+1,), thres)
    M += np.outer(v, np.ones(n))

    # M to K
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            d2 = np.sum((M[:, i] - M[:, j])**2)
            K[i, j] = np.exp(-d2 / 1000)

    if np.linalg.eigvals(K)[r] < 1e-4:
        break

print(np.linalg.eigvals(M.T @ M))
print(np.linalg.eigvals(K))
M,K

[7.96717541e+00 8.81407950e-01 2.89987656e-03 6.03550179e-16
 2.29337054e-17]
[4.99739184e+00 2.59600683e-03 1.06215427e-09 5.19683653e-07
 1.16285335e-05]


(array([[0.73425556, 0.        , 0.21991418, 1.44578374, 0.87697242],
        [0.73425556, 0.76869582, 0.67900565, 0.72225367, 0.77216503],
        [0.73425556, 0.73425556, 0.73425556, 0.73425556, 0.73425556]]),
 array([[1.        , 0.99945983, 0.99973244, 0.99949371, 0.9999782 ],
        [0.99945983, 1.        , 0.9999436 , 0.99790974, 0.9992312 ],
        [0.99973244, 0.9999436 , 1.        , 0.9984965 , 0.99955969],
        [0.99949371, 0.99790974, 0.9984965 , 1.        , 0.99967402],
        [0.9999782 , 0.9992312 , 0.99955969, 0.99967402, 1.        ]]))